In [1]:
import numpy as np
import os
import re
import pandas as pd
import pickle

from datetime import datetime

import tensorflow as tf
from tensorflow.keras import models, layers, Input
from tensorflow.keras import backend as K
from tensorflow.python.keras.callbacks import ModelCheckpoint

from sklearn.model_selection import train_test_split

### Wczytanie danych

In [2]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [3]:
placement_weights = np.array([1, 0.875, 0.75, 0.625, 0.375, 0.25, 0.125, 0])

In [4]:
with open('embeddings_100.pkl', 'rb') as file:
    weights = pickle.load(file)
word2idx = {}
with open('word2idx.pkl', 'rb') as file:
    word2idx = pickle.load(file)

In [6]:
print(word2idx)

{'TFT9_Augment_OverwhelmingForce': 1, 'TFT11_Augment_DryadCrown': 2, 'TFT5_Item_InfinityEdgeRadiant': 3, 'TFT9_Augment_Harmacist3': 4, 'TFT_Item_ThiefsGloves': 5, 'TFT11_Augment_ShenCarry': 6, 'TFT9_Augment_PumpingUp': 7, 'TFT_Augment_BranchingOut': 8, 'TFT11_Ornn': 9, 'TFT7_Item_ShimmerscaleMogulsMail': 10, 'TFT11_Augment_AtWhatCost': 11, 'TFT11_Augment_Warden': 12, 'TFT11_Item_HeavenlyEmblemItem': 13, 'TFT10_Augment_HeroicGrabBag': 14, 'TFT5_Item_QuicksilverRadiant': 15, 'TFT11_KhaZix': 16, 'TFT11_Augment_FatedCrest': 17, 'TFT11_Augment_ShenCarryPlus': 18, 'TFT_Item_Shroud': 19, 'TFT11_Augment_InvokerCrest': 20, 'TFT11_Augment_RainingGold': 21, 'TFT11_Darius': 22, 'TFT6_Augment_TheGoldenEgg': 23, 'TFT11_Augment_StoryweaverCrest': 24, 'TFT9_Augment_NotToday': 25, 'TFT11_Storyweaver7_AS': 26, 'TFT9_Augment_BalancedBudgetPlus': 27, 'TFT_Item_Zephyr': 28, 'TFT11_Alune': 29, 'TFT11_Augment_WardenCrest': 30, 'TFT_Item_SparringGloves': 31, 'TFT11_Item_InkshadowHorse': 32, 'TFT11_Augment_Alt

In [5]:
weights.shape

(451, 100)

In [6]:
sentences = []
with open('teams.txt', 'r') as file:
    for line in file:
        sentences.append(line.strip().split())

In [7]:
print(len(sentences), sentences[0])

67309 ['TFT11_Caitlyn', 'nan', 'nan', 'nan', 'TFT11_Jax', 'nan', 'nan', 'nan', 'TFT11_Senna', 'TFT5_Item_GuardianAngelRadiant', 'TFT_Item_HextechGunblade', 'TFT_Item_RapidFireCannon', 'TFT11_Shen', 'nan', 'nan', 'nan', 'TFT11_Illaoi', 'TFT_Item_IonicSpark', 'TFT_Item_FrozenHeart', 'TFT_Item_Crownguard', 'TFT11_Yone', 'nan', 'nan', 'nan', 'TFT11_Morgana', 'TFT_Item_AdaptiveHelm', 'nan', 'nan', 'TFT11_Kayn', 'TFT_Item_UnstableConcoction', 'TFT_Item_GuardianAngel', 'nan', 'TFT6_Augment_TinyTitans', 'TFT9_Augment_HighEndSector', 'TFT10_Augment_CrashTestDummies', '8']


In [8]:
maxWords = 0
for i in range(len(sentences)):
    if maxWords < len(sentences[i]):
        maxWords = len(sentences[i])

In [9]:
maxWords

64

In [10]:
X = np.zeros((len(sentences), maxWords - 1, 100))
Y = np.zeros((len(sentences)))
for i in range(len(sentences)):
    for j in range(len(sentences[i])):
        if sentences[i][j] == 'nan':
            X[i][j] = np.zeros((100,))
        else:
            if str(sentences[i][j]).isdigit():
                Y[i] = placement_weights[int(sentences[i][j]) - 1]
            else:
                X[i][j] = weights[word2idx[sentences[i][j]] - 1]

In [11]:
X.shape, Y.shape

((67309, 63, 100), (67309,))

In [12]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.1, random_state=42)

In [13]:
X_train.shape, X_test.shape, Y_train.shape, Y_test.shape 

((60578, 63, 100), (6731, 63, 100), (60578,), (6731,))

In [25]:
def create_data(X, Y):
    X1, X2, YY = [], [], []
    
    for i in range(len(X)):
        for j in range(len(X)):
            if i == j:
                continue
            X1.append(X[i])
            X2.append(X[j])
            
            if Y[i] == Y[j]:
                YY.append(0)
            elif Y[i] > Y[j]:
                YY.append(1)
            else:
                YY.append(-1)
            
    return np.array(X1), np.array(X2), np.array(YY)

In [ ]:
x1, x2, yy = create_data(X_train, Y_train)

In [ ]:
x1.shape, x2.shape, yy.shape

### Sieć syjamska

In [17]:
input_shape = (63, 100, 1)

In [18]:
def euclidean_distance(vects):
    x, y = vects
    sum_square = tf.reduce_sum(tf.square(x - y), axis=1, keepdims=True)
    return tf.sqrt(tf.maximum(sum_square, tf.keras.backend.epsilon()))

def contrastive_loss(y_true, y_pred, margin=1.0):
    """Contrastive loss function."""
    y_true = K.cast(y_true, 'float32')
    square_pred = K.square(y_pred)
    margin_square = K.square(K.maximum(margin - y_pred, 0))
    return K.mean((1 - y_true) * square_pred + y_true * margin_square)

In [19]:
# Base Network
base_network = models.Sequential(name='Base_Network')

base_network.add(Input(shape=input_shape))
base_network.add(layers.Conv2D(32, kernel_size=(2, 2), activation='relu'))
base_network.add(layers.MaxPooling2D((2, 2)))

base_network.add(layers.Conv2D(64, kernel_size=(2, 2), activation='relu'))
base_network.add(layers.MaxPooling2D((2, 2)))

base_network.add(layers.Flatten())
base_network.add(layers.Dropout(0.5))
base_network.add(layers.Dense(128))

# Siamese Network
input_a = Input(shape=input_shape, name='Team 1')
input_b = Input(shape=input_shape, name='Team 2')

processed_a = base_network(input_a)
processed_b = base_network(input_b)

distance = layers.Lambda(euclidean_distance, name='Euclidean_Distance')([processed_a, processed_b])

output = layers.Dense(1, activation='tanh', name='Output')(distance)

model_siamese = models.Model(inputs=[input_a, input_b], outputs=output, name='Siamese_Network')

# model_siamese.compile(loss=contrastive_loss, optimizer='adam', metrics=["accuracy"])
model_siamese.compile(loss='mse', optimizer='adam', metrics=["accuracy"])

model_siamese.summary()

Model: "Siamese_Network"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Team 1 (InputLayer)            [(None, 63, 100, 1)  0           []                               
                                ]                                                                 
                                                                                                  
 Team 2 (InputLayer)            [(None, 63, 100, 1)  0           []                               
                                ]                                                                 
                                                                                                  
 Base_Network (Sequential)      (None, 128)          2957664     ['Team 1[0][0]',                 
                                                                  'Team 2[0][0]']   

In [20]:
# model_siamese.fit([x1, x2], yy, epochs=1000, batch_size=64, shuffle=True, verbose=True, validation_split=0.15)

Epoch 1/1000
403/403 [==============================] - 11s 18ms/step - loss: 0.8785 - accuracy: 0.1265 - val_loss: 0.8751 - val_accuracy: 0.1272
Epoch 2/1000
403/403 [==============================] - 7s 17ms/step - loss: 0.8742 - accuracy: 0.1265 - val_loss: 0.8736 - val_accuracy: 0.1272
Epoch 3/1000
403/403 [==============================] - 7s 17ms/step - loss: 0.8737 - accuracy: 0.1265 - val_loss: 0.8732 - val_accuracy: 0.1272
Epoch 4/1000
403/403 [==============================] - 7s 17ms/step - loss: 0.8736 - accuracy: 0.1265 - val_loss: 0.8729 - val_accuracy: 0.1272
Epoch 5/1000
403/403 [==============================] - 7s 17ms/step - loss: 0.8733 - accuracy: 0.1265 - val_loss: 0.8729 - val_accuracy: 0.1272
Epoch 6/1000
403/403 [==============================] - 7s 17ms/step - loss: 0.8731 - accuracy: 0.1265 - val_loss: 0.8726 - val_accuracy: 0.1272
Epoch 7/1000
403/403 [==============================] - 7s 17ms/step - loss: 0.8718 - accuracy: 0.1265 - val_loss: 0.8730 - val_a

In [22]:
model_siamese.save(f'siamese.h5')

In [23]:
x1_val, x2_val, y_val = create_data(X_test, Y_test)

In [24]:
model_siamese.evaluate([x1_val, x2_val], y_val, verbose=True)

106/106 [==============================] - 1s 5ms/step - loss: 1.8680 - accuracy: 0.4333


[1.8680330514907837, 0.43328380584716797]